In [2]:
import websocket
import json
# import pprint
#import talib
import numpy
import pandas as pd
import src.credentials
import src.binance_functions

# import config
from binance.client import Client
from binance.enums import *

SOCKET = "wss://stream.binance.com:9443/ws/btcbrl@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 65
TRADE_SYMBOL = 'BTCBRL'
TRADE_QUANTITY = 0.05

client = Client(credentials.app_key, credentials.app_secret, tld='us')


In [3]:
response = binance_functions.send_signed_request("GET", "/api/v3/account")

GET https://api.binance.com/api/v3/account?timestamp=1708379876078&signature=42990f8944a50d8781e4ffcf99c5095a48809fbc99d762268fb0749583b3630c


In [4]:
response

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1708192826264,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [3]:
df_balances = pd.DataFrame(response['balances'])
df_balances['free'] = df_balances['free'].astype(float)
df_balances['locked'] = df_balances['locked'].astype(float)
df_balances[(df_balances['free'] + df_balances['locked']) > 0]


,asset,free,locked
208,BRL,50.0,0.0


In [4]:
brl_balance = int(df_balances[df_balances['asset'] == 'BRL']['free'])
type(int(brl_balance))

int

* DEFINIR TRADE_QUANTITY
* GUARDAR TRANSAÇÕES EM ALGUM DATAFRAME


In [ ]:
closes = []
in_position = False

df_orders = pd.DataFrame(columns=[''])

client = Client(credentials.API_KEY, credentials.API_SECRET, tld='us')

baseline = 50

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        # order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
        
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    # print('received message')
    json_message = json.loads(message)
    # pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {} ' --- closes: ', {}".format(close, len(closes)))
        closes.append(float(close))
        print("closes")
        print(' --- closes: ', len(closes))

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))


            if last_rsi > RSI_OVERBOUGHT and in_position:
                
                if in_position:
                
                    # get BTC free balance
                    btc_balance = binance_functions.get_balance('BTC')
                    
                    # convert BTC balance to BRL
                    btcbrl_balance = close * btc_balance

                    # get profit compared to baseline
                    profit_value = btcbrl_balance - baseline
                    
                    if profit_value > 0:

                        # convert profit in BRL to BTC
                        TRADE_QUANTITY = profit_value / close
                        
                        # create sell order
                        order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)                    
                        if order_succeeded:
                            in_position = False 

                        in_position = False
                    
                    else:

                        print('In position to sell, but there''s no profit over the baseline')
                else:
                    # df.at[i, 'position'] = 'DOI'
                    print('In position to sell, but don''t on it yet')
                    
            elif last_rsi < RSI_OVERSOLD:
                
                if in_position:
                    print('In position to buy, but already have it')
                    # df.at[i, 'position'] = 'HOLD'
                else:
                    
                    # get BRL free balance
                    brl_balance = binance_functions.get_balance('BRL')

                    if brl_balance > 0:
                
                        # convert BRL balance to BTC
                        TRADE_QUANTITY = brl_balance / close
                        
                        # create buy order
                        order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = True

                        # df.at[i, 'position'] = 'BOUGHT'
                        in_position = True

                    else:
                        
                        print('In position to buy, but don''t have enough balance')
            else:
                #RSI entre overbought e oversold
                print('No position to take')
                # df.at[i, 'position'] = 'HOLD'




In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()